# Random Forest with genotype -> phenotype data

In [ ]:
import pandas as pd
import numpy as np
import os

In [ ]:
import sklearn
import random
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.model_selection import KFold

# Importing the data

In [ ]:
# Config for accessing the data on the s3 storage
storage_options = {'anon':True, 'client_kwargs':{'endpoint_url':'https://os.unil.cloud.switch.ch'}}
s3_path = 's3://lts2-graphnex/BXDmice/'

In [ ]:
# Load the data
genotype_path = os.path.join(s3_path, 'geno_reduced.csv.gz')
#genotype_path = os.path.join(s3_path, 'genotype_BXD.csv.gz')
genotype = pd.read_csv(genotype_path, storage_options=storage_options)
print('File {} Opened.'.format(genotype_path))
phenotype_path = os.path.join(s3_path, 'Phenotype.txt.gz')
phenotype = pd.read_csv(phenotype_path, sep='\t', storage_options=storage_options)
print('File {} Opened.'.format(phenotype_path))
# Phenotype description
phenotypeinfo_path = os.path.join(s3_path, 'phenotypes_id_aligner.txt.gz')
phenotypeinfo = pd.read_csv(phenotypeinfo_path, sep='\t', storage_options=storage_options)
print('File {} Opened.'.format(phenotypeinfo_path))

In [ ]:
# Random Forest function
def run_RF(data,labels, n_estimators=1000, max_depth=3, nb_folds=3):
    # Run k-fold learning
    # with random forest
    k_fold = KFold(nb_folds)
    kscores =np.zeros(nb_folds)
    #print(kscores)
    for k, (train, test) in enumerate(k_fold.split(data, labels)):
        clf = RandomForestRegressor(n_estimators=n_estimators, max_depth=max_depth)
        clf.fit(data[train], labels[train])
        score = clf.score(data[test], labels[test])
        kscores[k] = score
    # The final score is the mean of all the k fold scores
    meanscore = np.mean(kscores)
    feat = clf.feature_importances_ # TODO take the features of the best fold or of all the folds
    # nb of best features to record
    nb_features = 5
    feat_index = np.argsort(feat)[-nb_features:]
    feat_index = feat_index[::-1]
    feat_score = feat[feat_index]
    return meanscore,feat_index,feat_score

## Example on one phenotype

In [ ]:
pheno_id = 'X122'
phenotype[phenotype['PhenoID']==pheno_id]

In [ ]:
phenotypeinfo[phenotypeinfo['PhenoID']=='X122']

In [ ]:
experiment = phenotype[phenotype['PhenoID']==pheno_id]
pheno_labels = experiment.loc[:,'C57BL.6J':].dropna(axis=1)
geno_data = genotype[pheno_labels.keys()]
score,feat_index,feat_score = run_RF(geno_data.values.T, pheno_labels.values.ravel(), 2000, 6, 3)    
print('Score',score)
print('Best features {}, best feature scores {}'.format(feat_index, feat_score))

## Loop on all the phenotypes

In [ ]:
for pheno_index, pheno_row in phenotype.iterrows():
    pheno_id = pheno_row['PhenoID']
    pheno_labels = pheno_row['C57BL.6J':].dropna()
    if len(pheno_labels)<5: # Skip small list of samples
        print(pheno_id, pheno_labels)
        continue
    geno_data = genotype[pheno_labels.keys()]
    score,feat_index,feat_score = run_RF(geno_data.values.T, pheno_labels.values, 300, 3, 2)    
    print(score)
    #if score > 0.8:
    #    print(score, feat_index, feat_score)
    #if pheno_index%10 ==0:
    #    print(pheno_index)